# Agents

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true" # Langsmith tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

## Create the Wikipedia Wrapper Source 1

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_wrapper = WikipediaAPIWrapper(top_k_results = 1, doc_content_chars_max = 200)
wiki_tool = WikipediaQueryRun(api_wrapper = wiki_wrapper)

## Create the custom Retriever and retirever to search for agents - SOurce 2

In [3]:
# Web Based Loader
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import bs4
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# load, chunk and index the content of the htm page

loader = WebBaseLoader(web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                       bs_kwargs = dict(parse_only = bs4.SoupStrainer(
                           class_ = ("post-title", "post-content", "post_header")
                       )))
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = text_splitter.split_documents(documents)
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.
d:\Softwares\Anaconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000246F42B8750>)

In [4]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "agent_search", "Search for information about agents. For any question about agents use this tool")
retriever_tool.name

'agent_search'

## Create the arxiv Wrapper Source 3

In [5]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arx_wrapper = ArxivAPIWrapper(top_k_results = 1, doc_content_chars_max = 200)
arx_tool = ArxivQueryRun(api_wrapper = arx_wrapper)

## Combine all the tools

In [6]:
tools = [retriever_tool, wiki_tool, arx_tool]

### Create Agents to query from the tool - LLM + Prompt + tools

In [7]:
# ABuild llm
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = 'gpt-3.5-turbo', temperature = 0)




In [8]:
# Build Prompt
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [9]:
# Build Agent
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [10]:
# Agent Executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True) 
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [11]:
# invoke the agent
agent_executor.invoke({"input": "What are agents ?"})



> Entering new AgentExecutor chain...

Invoking: `agent_search` with `{'query': 'agents'}`


They also discussed the risks, especially with illicit drugs and bioweapons. They developed a test set containing a list of known chemical weapon agents and asked the agent to synthesize them. 4 out of 11 requests (36%) were accepted to obtain a synthesis solution and the agent attempted to consult documentation to execute the procedure. 7 out of 11 were rejected and among these 7 rejected cases, 5 happened after a Web search while 2 were rejected based on prompt only.
Generative Agents Simulation#
Generative Agents (Park, et al. 2023) is super fun experiment where 25 virtual characters, each controlled by a LLM-powered agent, are living and interacting in a sandbox environment, inspired by The Sims. Generative agents create believable simulacra of human behavior for interactive applications.
The design of generative agents combines LLM with memory, planning and reflection mechanisms to enabl

{'input': 'What are agents ?',
 'output': 'Agents can refer to various entities depending on the context. In the context of your query, agents can be artificial intelligence systems or programs that are designed to perform specific tasks or make decisions on behalf of users. These agents can range from simple automated scripts to complex AI algorithms that can learn and adapt.\n\nIf you need more specific information about a particular type of agent or have any other questions related to agents, feel free to ask!'}

In [12]:
agent_executor.invoke({"input": "What is dragonball ?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Dragon Ball'}`


Page: Dragon Ball
Summary: Dragon Ball (Japanese: ドラゴンボール, Hepburn: Doragon Bōru) is a Japanese media franchise created by Akira Toriyama in 1984. The initial manga, written and illustrated by ToriyamDragon Ball is a Japanese media franchise created by Akira Toriyama in 1984. It includes a manga series, anime series, films, video games, and more. The story follows the adventures of Son Goku from his childhood through adulthood as he trains in martial arts and explores the world in search of the seven orbs known as the Dragon Balls.

> Finished chain.


{'input': 'What is dragonball ?',
 'output': 'Dragon Ball is a Japanese media franchise created by Akira Toriyama in 1984. It includes a manga series, anime series, films, video games, and more. The story follows the adventures of Son Goku from his childhood through adulthood as he trains in martial arts and explores the world in search of the seven orbs known as the Dragon Balls.'}